In [37]:
import tensorflow as tf

In [38]:
# get the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(
    "/tmp/tensorflow/mnist/input_data", 
    reshape=False
)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [39]:
# wrap value function
# for label and image infomation
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# for image it self
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [40]:
# construct the full path to store data
def _data_path(data_directory,name):
    if not os.path.isdir(data_directory):
        os.makedirs(data_directory)
    return os.path.join(data_directory,f'{name}.tfrecords')

In [41]:
import os
import sys
import numpy as np

In [42]:
def convert_to(images,labels,name,data_direcotory,num_shards=1):
    
    num_examples, rows, cols, depth = images.shape
    data_set = list(zip(images,labels))
    
    def _process_examples(example_dataset,filename):
        print(f'Processing {filename} data')
        dataset_length = len(example_dataset)
        with tf.python_io.TFRecordWriter(filename) as writer:
            for index, (image, label) in enumerate(example_dataset):
                sys.stdout.write(f"\rProcessing sample {index+1} of {dataset_length}")
                sys.stdout.flush()

                image_raw = image.tostring()
                example = tf.train.Example(features=tf.train.Features(feature={
                    'height': _int64_feature(rows),
                    'width': _int64_feature(cols),
                    'depth': _int64_feature(depth),
                    'label': _int64_feature(int(label)),
                    'image_raw': _bytes_feature(image_raw)
                }))
                writer.write(example.SerializeToString())
            print()
    
    if num_shards == 1:
        _process_examples(data_set, _data_path(data_directory, name))
    else:
        sharded_dataset = np.array_split(data_set, num_shards)
        for shard, dataset in enumerate(sharded_dataset):
            _process_examples(dataset, _data_path(data_directory, f'{name}-{shard+1}'))

In [43]:
images = np.concatenate([mnist.train.images,mnist.validation.images,mnist.test.images],0)

In [44]:
labels = np.concatenate([mnist.train.labels,mnist.validation.labels,mnist.test.labels],0)

In [45]:
data_directory = "mnist"
convert_to(images,labels, 'data', data_directory, num_shards=12)

Processing mnist/data-1.tfrecords data
Processing sample 5834 of 5834
Processing mnist/data-2.tfrecords data
Processing sample 5834 of 5834
Processing mnist/data-4.tfrecords data
Processing sample 5834 of 5834
Processing mnist/data-5.tfrecords data
Processing sample 5833 of 5833
Processing mnist/data-7.tfrecords data
Processing sample 5833 of 5833
Processing mnist/data-8.tfrecords data
Processing sample 5833 of 5833
Processing mnist/data-9.tfrecords data
Processing sample 5833 of 5833
Processing mnist/data-10.tfrecords data
Processing sample 5833 of 5833
Processing mnist/data-11.tfrecords data
Processing sample 5833 of 5833
Processing mnist/data-12.tfrecords data
Processing sample 5833 of 5833
